# 국민동의청원 분류하기

본 프로젝트는 국민동의청원 사이트에서 참여 인원이 높은 청원 글들의 특징을 학습하여, 새로운 청원 글이 입력되었을 때 학습된 글들과 유사성을 계산하여 주목 받을 만한 글인지 아닌지를 판단하도록 합니다.

국민동의청원 사이트 [링크](https://petitions.assembly.go.kr)

## 1. 크롤링

[원본 데이터 추출]

In [ ]:
import requests
import pandas as pd

url = 'https://petitions.assembly.go.kr/api/petits?pageIndex=1&recordCountPerPage=10000&sort=AGRE_CO-&searchCondition=sj&searchKeyword=&petitRealmCode=&sttusCode=PETIT_FORMATN,CMIT_FRWRD,PETIT_END&resultCode=BFE_OTHBC_WTHDRAW,PROGRS_WTHDRAW,PETIT_UNACPT,APPRVL_END_DSUSE,ETC_TRNSF&notInColumn=RESULT_CODE&beginDate=20200101&endDate=20240731&ageCd='

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)
response.raise_for_status()

response_body = response.json()

petitions = response_body
df_orignal = pd.DataFrame(petitions)

df_orignal.head()

[학습 데이터 추출 및 저장]

In [ ]:
# 학습에 필요한 데이터 항목을 추출합니다.

# 크롤링 원본 데이터에서 딥러닝 할 dataset 컬럼 패핑
colums_to_extract = {
    'rowNum': 'rowno',
    'petitRealmNm': 'category',
    'petitSj': 'title',
    'petitCn': 'content',
    'agreCo': 'count',
    'agreBeginDe': 'start',
    'agreEndDe': 'end'
}

# 원본 데이터에서 특정 열을 추출하고 열 이름 변경하여 새로운 df에 할당
df_extracted = df_orignal[list(colums_to_extract.keys())].rename(columns=colums_to_extract)

# 날짜 형식 변환(원본 yyyy-MM-dd 24HH:MI:SS, 변환 yyyy-MM-dd)
df_extracted['start'] = pd.to_datetime(df_extracted['start']).dt.strftime('%Y-%m-%d')
df_extracted['end'] = pd.to_datetime(df_extracted['end']).dt.strftime('%Y-%m-%d')

df_extracted.head()

In [ ]:
dataset_file_path = 'data/petitions-assembly-dataset.csv'
df_extracted.to_csv(dataset_file_path, index=False)

print(f"추출된 데이터가 {dataset_file_path}에 저장되었습니다.")

## 2. 학습 데이터 전처리

In [ ]:
# 학습 데이터 로드
df = pd.read_csv(dataset_file_path)

In [ ]:
# 전처리 전 출력
df.loc[1]['content']

In [ ]:
import re 

def remove_white_space(text):
    return re.sub(r'[\t\r\n\f\v]', ' ', str(text))

def remove_special_char(text):
    return re.sub('[^ ㄱ-ㅣ가-힣 0-9]+', ' ', str(text))

# title 전처리
df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

# content 전처리
df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

In [ ]:
# 전처리 후 출력
df.loc[1]['content']

## 3. 토크나이징 및 변수 생성

[토크나이징]

In [ ]:
from konlpy.tag import Okt
import platform

try:
    okt = Okt()
except:
    print('JAVA_HOME not found.')

    # get platform
    platform_name = platform.system()
    print(f'Platform name : {platform_name}')

    if platform_name=='Darwin':
        JVM_PATH = '/Library/Java/JavaVirtualMachines/zulu-1.8.0.jdk/Contents/Home/jre/lib/jli/libjli.dylib'
    else:
        JVM_PATH = 'C:\\Library\\Java\\jdk-17.0.12\\bin\\server\\jvm.dll'
    okt = Okt(JVM_PATH)

df['title_token'] = df.title.apply(okt.morphs)
print('title_token completed.')
df['content_token'] = df.content.apply(okt.nouns)
print('content_token completed.')

[파생변수 생성]

In [ ]:
df['token_final'] = df.title_token + df.content_token

# 국민동의청원에서는 int형이므로 생략함.
# df['count'] = df['count'].replace({',' : ''}, regex = True).apply(lambda x : int(x))

print(df.dtypes)

df['label'] = df['count'].apply(lambda x: 'Yes' if x>=1000 else 'No')

In [ ]:
df_drop = df[['token_final', 'label']]

df_drop.head()

[데이터 엑셀로 저장]

In [ ]:
df_drop_path = 'data/df_drop.csv'
df_drop.to_csv(df_drop_path, index=False, encoding='utf-8-sig')

print(f"토큰 데이터가 {df_drop_path}에 저장되었습니다.")

## 4. 단어 임베딩

[단어 임베딩]

In [ ]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(df_drop['token_final'],
                           sg=1,
                           vector_size=100,
                           window=2,
                           min_count=1,
                           workers=4
                           )

print(embedding_model)

model_result = embedding_model.wv.most_similar("탄핵")
print(model_result)

[임베드 모델 저장 및 로드]

In [ ]:
from gensim.models import KeyedVectors

tokens_w2v_path = 'data/petitions_tokens_w2v.pkl'

# 모델 저장
embedding_model.wv.save_word2vec_format(tokens_w2v_path) 
# 모델 로드
loaded_model = KeyedVectors.load_word2vec_format(tokens_w2v_path)

model_result = loaded_model.most_similar("탄핵")
print(model_result)

## 5. 실험 설계

[데이터셋 분할 및 저장]

In [ ]:
from numpy.random import RandomState

rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[(~df_drop.index.isin(tr.index))]

train_file_path = 'data/train.csv'
validation_file_path = 'data/validation.csv'
tr.to_csv(train_file_path, index=False, encoding='utf-8-sig')
val.to_csv(validation_file_path, index=False, encoding='utf-8-sig')

[Field 클래스 정의]

In [ ]:
import torchtext 
from torchtext.data import Field

def tokenizer(text):
    return re.sub('[\[\]\']', '', str(text)).split(', ')

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential=False)

[데이터 불러오기]

In [ ]:
from torchtext.data import TabularDataset

train, validation = TabularDataset.splits(
    path='data/',
    train='train.csv',
    validation='validation.csv',
    format='csv',
    fields=[('text', TEXT), ('label', LABEL)],
    skip_header=True
)

print(f'Train: {train[0].text}, {train[0].label}')
print(f'Validation: {validation[0].text}, {validation[0].label}')

[단어장 및 DataLoader 정의]

In [ ]:
import torch
from torchtext.vocab import Vectors
from torchtext.data import BucketIterator

vectors = Vectors(name=tokens_w2v_path)

TEXT.build_vocab(train, vectors=vectors, min_freq=1, max_size=None)
LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(
    datasets=(train, validation),
    batch_sizes=8,
    device=device,
    sort=False
)

print(f'임베팅 벡터의 개수와 차원 {TEXT.vocab.vectors.shape}')